# 概述
改进数解法    
干道协调，对称放行，一个7个路口，ABCDEFG    
间距：300，250，380，450，300，210    
绿信比：0.5, 0.55, 0.6, 0.6, 0.5, 0.7, 0.55    
公共信号周期范围：40-120s    
速度：12m/s (43.2km/h)  

In [7]:
import pandas as pd
import numpy as np
import math

# parameters
node_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
spacing = [300, 250, 380, 450, 300, 210]
ratio = [0.5, 0.55, 0.6, 0.6, 0.5, 0.7, 0.55]    # green time ratio
c_min = 40
c_max = 120
step = 1    # idea distance will iterate with step
v = 12    # unit: m/s

In [8]:
idea_dis_min = math.ceil(0.5 * v * c_min)
idea_dis_max = math.floor(0.5 * v * c_max)
print("idea distance in: {} to {}".format(idea_dis_min, idea_dis_max))

idea distance in: 240 to 720


# 最佳间距及周期

In [9]:
cols = ['idea_dis'] + node_list + ['b', 'b/a', 'b_index']
dis_diff = pd.DataFrame(columns=cols)

dis = [0] + spacing
dis_pre = [0]
for i in range(1, len(dis)):
    dis_pre.append(dis_pre[-1] + dis[i])
print(dis_pre)

for i in range(idea_dis_min, idea_dis_max+step, step):
    tmp = []
    for j in dis_pre:
        tmp.append(j % i)
        
    #print(tmp)

    tmp_sorted = sorted(tmp)
    tmp_sorted.append(i)
    #print(tmp_sorted)

    tmp_diff = []
    for j in range(1, len(tmp_sorted)):
        tmp_diff.append(tmp_sorted[j] - tmp_sorted[j-1])
    #tmp_diff.append(tmp_sorted[0] - tmp_sorted[-1])
    #print(tmp_diff)

    diff_max = max(tmp_diff)
    diff_max_id = tmp_diff.index(diff_max)
    #print(diff_max)

    row = [i] + tmp + [diff_max, (diff_max / i), diff_max_id]
    #print(row)

    dis_diff.loc[len(dis_diff.index)] = row 

print(dis_diff)

dis_diff_max = dis_diff[dis_diff['b/a'] == dis_diff['b/a'].max()]
dis_diff_max = dis_diff_max.reset_index(drop=True)
print(dis_diff_max)

dis_opt = dis_diff_max.at[0, 'idea_dis']

c_opt = 2 * dis_opt / v
print(c_opt)

[0, 300, 550, 930, 1380, 1680, 1890]
     idea_dis    A      B      C      D      E      F      G      b       b/a  \
0       240.0  0.0   60.0   70.0  210.0  180.0    0.0  210.0  110.0  0.458333   
1       241.0  0.0   59.0   68.0  207.0  175.0  234.0  203.0  107.0  0.443983   
2       242.0  0.0   58.0   66.0  204.0  170.0  228.0  196.0  104.0  0.429752   
3       243.0  0.0   57.0   64.0  201.0  165.0  222.0  189.0  101.0  0.415638   
4       244.0  0.0   56.0   62.0  198.0  160.0  216.0  182.0   98.0  0.401639   
..        ...  ...    ...    ...    ...    ...    ...    ...    ...       ...   
476     716.0  0.0  300.0  550.0  214.0  664.0  248.0  458.0  214.0  0.298883   
477     717.0  0.0  300.0  550.0  213.0  663.0  246.0  456.0  213.0  0.297071   
478     718.0  0.0  300.0  550.0  212.0  662.0  244.0  454.0  212.0  0.295265   
479     719.0  0.0  300.0  550.0  211.0  661.0  242.0  452.0  211.0  0.293463   
480     720.0  0.0  300.0  550.0  210.0  660.0  240.0  450.0  210.0  0.2

# 实际交叉口对应理想交叉口的编号和方位

In [10]:
dis_diff_opt = dis_diff_max.loc[0]
print(dis_diff_opt)

ideas_count = int(np.ceil(dis_pre[-1] / dis_diff_opt['idea_dis']) + 1)
print("count: {}".format(ideas_count))

id_reals_to_ideas = []    # the id of ideas matched to reals
pos_reals_to_ideas = []    # 0 means real is on the left, 1 means right
dis_reals_to_ideas = []

list_dis_diff_opt = dis_diff_opt[1:-3]
list_dis_diff_opt = list(list_dis_diff_opt.values)
list_dis_diff_opt.sort()

x = (dis_diff_opt['idea_dis'] - dis_diff_opt['b']) / 2
y = list_dis_diff_opt[int(dis_diff_opt['b_index'])]
offset = x - y    # offset for dis_pre
dis_pre_offseted = [i + offset for i in dis_pre]
#print("offseted: {}".format(dis_pre_offseted))

for i in range(len(dis_pre_offseted)):
    dis_real_to_ideas = []
    dis_abs = []

    for j in range(ideas_count):
        tmp = dis_pre_offseted[i] - dis_diff_opt['idea_dis']*j
        dis_real_to_ideas.append(tmp)
        dis_abs.append(np.abs(tmp))
    #print(dis_real_to_ideas)

    dis_min = np.min(dis_abs)
    dis_min_index = dis_abs.index(dis_min)
    #print("id: {}, val: {}".format(dis_min_index, dis_min))

    id_reals_to_ideas.append(dis_min_index)
    dis_reals_to_ideas.append(dis_real_to_ideas[dis_min_index])

    if dis_real_to_ideas[dis_min_index] < 0:
        pos_reals_to_ideas.append(0)
    else:
        pos_reals_to_ideas.append(1)

print(id_reals_to_ideas)
print(pos_reals_to_ideas)
print(dis_reals_to_ideas)

idea_dis    270.000000
A             0.000000
B            30.000000
C            10.000000
D           120.000000
E            30.000000
F            60.000000
G             0.000000
b           150.000000
b/a           0.555556
b_index       6.000000
Name: 0, dtype: float64
count: 8
[0, 1, 2, 3, 5, 6, 7]
[0, 0, 0, 1, 0, 1, 0]
[-60.0, -30.0, -50.0, 60.0, -30.0, 0.0, -60.0]


# 损失，相位差

In [11]:
# lost green time ratio
loss = [np.abs(i / dis_diff_opt['idea_dis']) for i in dis_reals_to_ideas]
loss

offset_ratio = [i / 2 for i in loss]    # offset green time ratio

phase_diff = []
for i in range(len(id_reals_to_ideas)):
    if id_reals_to_ideas[i] % 2 == 0:
        phase_diff.append(0.5 - 0.5*ratio[i])
    else:
        phase_diff.append(1 - 0.5*ratio[i])

up_ratio = []
down_ratio = []
for i in range(len(ratio)):
    if pos_reals_to_ideas[i] == 1:
        up_ratio.append(0.5*ratio[i] - offset_ratio[i])
        down_ratio.append(0.5*ratio[i] + offset_ratio[i])
    else:
        up_ratio.append(0.5*ratio[i] + offset_ratio[i])
        down_ratio.append(0.5*ratio[i] - offset_ratio[i])

width = min(up_ratio) + min(down_ratio)

print("loss: {}".format(loss))
print("offset ratio: {}".format(offset_ratio))
print("phase different: {}".format(phase_diff))
print("up: {}".format(up_ratio))
print("down: {}".format(down_ratio))
print("width: {}".format(width))

loss: [0.2222222222222222, 0.1111111111111111, 0.18518518518518517, 0.2222222222222222, 0.1111111111111111, 0.0, 0.2222222222222222]
offset ratio: [0.1111111111111111, 0.05555555555555555, 0.09259259259259259, 0.1111111111111111, 0.05555555555555555, 0.0, 0.1111111111111111]
phase different: [0.25, 0.725, 0.2, 0.7, 0.75, 0.15000000000000002, 0.725]
up: [0.3611111111111111, 0.3305555555555556, 0.3925925925925926, 0.18888888888888888, 0.3055555555555556, 0.35, 0.3861111111111111]
down: [0.1388888888888889, 0.21944444444444447, 0.2074074074074074, 0.4111111111111111, 0.19444444444444445, 0.35, 0.16388888888888892]
width: 0.3277777777777778


# 结果展示

In [12]:
idx = ['ideal intersection id', 'position', 'green time ratio', 'phase diffrernce', 'lost ratio', 'offset ratio', 
        'ratio on the center line', 'ratio down the center line']

pos_char = []
for i in pos_reals_to_ideas:
    if i == 0:
        pos_char.append('left')
    else:
        pos_char.append('right')

print('Optimal cycle: {}'.format(c_opt))
print('Band width: {}'.format(width))
output_table = [id_reals_to_ideas, pos_char, ratio, phase_diff, loss, offset_ratio, up_ratio, down_ratio]
df_output = pd.DataFrame(output_table, columns=node_list, index=idx)
df_output

Optimal cycle: 45.0
Band width: 0.3277777777777778


,A,B,C,D,E,F,G
ideal intersection id,0,1,2,3,5,6,7
position,left,left,left,right,left,right,left
green time ratio,0.5,0.55,0.6,0.6,0.5,0.7,0.55
phase diffrernce,0.25,0.725,0.2,0.7,0.75,0.15,0.725
lost ratio,0.222222,0.111111,0.185185,0.222222,0.111111,0.0,0.222222
offset ratio,0.111111,0.055556,0.092593,0.111111,0.055556,0.0,0.111111
ratio on the center line,0.361111,0.330556,0.392593,0.188889,0.305556,0.35,0.386111
ratio down the center line,0.138889,0.219444,0.207407,0.411111,0.194444,0.35,0.163889


# 封装版本

In [ ]:
class LineCoordination:
    def __init__(self, dis_list):

        self.dis_list = dis_list
        self.index_list = [i for i in range(len(dis_list))]
        cols = ['idea_dis'] + self.index_list + ['b', 'b/a', 'b_index']
        self.dis_diff = pd.DataFrame(columns=cols)

        self.dis_pre = [0]
        for i in range(1, len(self.dis_list)):
            self.dis_pre.append(self.dis_pre[-1] + dis_list[i])
        #print(dis_pre)

        self.optimal_cycle = 0
        self.dis_diff_opt = None

    def findOptimal(self):

        self.dis_diff.drop(index=self.dis_diff.index)
        self.dis_diff_opt = None

        for i in range(idea_dis_min, idea_dis_max+step, step):
            tmp = []
            for j in self.dis_pre:
                tmp.append(j % i)
        
            #print(tmp)

            tmp_sorted = sorted(tmp)
            tmp_sorted.append(i)
            #print(tmp_sorted)

            tmp_diff = []
            for j in range(1, len(tmp_sorted)):
                tmp_diff.append(tmp_sorted[j] - tmp_sorted[j-1])
            #tmp_diff.append(tmp_sorted[0] - tmp_sorted[-1])
            #print(tmp_diff)

            diff_max = max(tmp_diff)
            diff_max_id = tmp_diff.index(diff_max)
            #print(diff_max)

            row = [i] + tmp + [diff_max, (diff_max / i), diff_max_id]
            #print(row)

            self.dis_diff.loc[len(self.dis_diff.index)] = row 

        dis_diff_max = self.dis_diff[self.dis_diff['b/a'] == self.dis_diff['b/a'].max()]
        dis_diff_max = dis_diff_max.reset_index(drop=True)
        self.dis_diff_opt = dis_diff_max.loc[0]

        dis_opt = dis_diff_max.at[0, 'idea_dis']
        self.optimal_cycle = 2 * dis_opt / v

        return self.optimal_cycle, self.dis_diff, self.dis_diff_opt

    def getOptimalInfo(self, ratio):

        if self.dis_diff_opt.empty:
            print("Error! optimal has not been found. Please run findOptimal. ")
            return None, None

        ideas_count = int(np.ceil(self.dis_pre[-1] / self.dis_diff_opt['idea_dis']) + 1)

        id_reals_to_ideas = []    # the id of ideas matched to reals
        pos_reals_to_ideas = []    # 0 means real is on the left, 1 means right
        dis_reals_to_ideas = []

        list_dis_diff_opt = self.dis_diff_opt[1:-3]
        list_dis_diff_opt = list(list_dis_diff_opt.values)
        list_dis_diff_opt.sort()

        x = (self.dis_diff_opt['idea_dis'] - self.dis_diff_opt['b']) / 2
        y = list_dis_diff_opt[int(self.dis_diff_opt['b_index'])]
        offset = x - y    # offset for dis_pre
        dis_pre_offseted = [i + offset for i in self.dis_pre]
        #print("offseted: {}".format(dis_pre_offseted))

        for i in range(len(dis_pre_offseted)):
            dis_real_to_ideas = []
            dis_abs = []

            for j in range(ideas_count):
                tmp = dis_pre_offseted[i] - self.dis_diff_opt['idea_dis']*j
                dis_real_to_ideas.append(tmp)
                dis_abs.append(np.abs(tmp))
            #print(dis_real_to_ideas)

            dis_min = np.min(dis_abs)
            dis_min_index = dis_abs.index(dis_min)
            #print("id: {}, val: {}".format(dis_min_index, dis_min))

            id_reals_to_ideas.append(dis_min_index)
            dis_reals_to_ideas.append(dis_real_to_ideas[dis_min_index])

            if dis_real_to_ideas[dis_min_index] < 0:
                pos_reals_to_ideas.append(0)
            else:
                pos_reals_to_ideas.append(1)

        # lost green time ratio
        loss = [np.abs(i / self.dis_diff_opt['idea_dis']) for i in dis_reals_to_ideas]

        offset_ratio = [i / 2 for i in loss]    # offset green time ratio

        phase_diff = []
        for i in range(len(id_reals_to_ideas)):
            if id_reals_to_ideas[i] % 2 == 0:
                phase_diff.append(0.5 - 0.5*ratio[i])
            else:
                phase_diff.append(1 - 0.5*ratio[i])

        up_ratio = []
        down_ratio = []
        for i in range(len(ratio)):
            if pos_reals_to_ideas[i] == 1:
                up_ratio.append(0.5*ratio[i] - offset_ratio[i])
                down_ratio.append(0.5*ratio[i] + offset_ratio[i])
            else:
                up_ratio.append(0.5*ratio[i] + offset_ratio[i])
                down_ratio.append(0.5*ratio[i] - offset_ratio[i])

        width = min(up_ratio) + min(down_ratio)

        idx = ['idea junction id', 'position', 'green time ratio', 'phase diffrernce', 'lost ratio', 'offset ratio', 
        'ratio on the center line', 'ratio down the center line']

        pos_char = []
        for i in pos_reals_to_ideas:
            if i == 0:
                pos_char.append('left')
            else:
                pos_char.append('right')

        output_table = [id_reals_to_ideas, pos_char, ratio, phase_diff, loss, offset_ratio, up_ratio, down_ratio]
        df_output = pd.DataFrame(output_table, columns=self.index_list, index=idx)

        return width, df_output

# testcase
test = [0, 300, 250, 380, 450, 300, 210]
ratio = [0.5, 0.55, 0.6, 0.6, 0.5, 0.7, 0.55]
line = LineCoordination(test)
opt_cycle, diff, diff_opt = line.findOptimal()
width, output = line.getOptimalInfo(ratio)
print("cycle: {}".format(opt_cycle))
print(diff)
print(diff_opt)
print("Band width: {}".format(width))
print(output)